In [2]:

import time
import pandas as pd
import re
import numpy as np
from konlpy.tag import Kkma, Okt
from sklearn.model_selection import train_test_split
import kss
import time
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/home/bigdata/jupyter_notebook/wontae/NanumGothic.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location)
plt.rc('font', family = font_name.get_name())
%matplotlib inline

In [3]:
import os
import re
from keras import regularizers
import keras
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Dense, Input, RepeatVector, Permute, Multiply, Lambda,BatchNormalization
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed, LSTM, Dropout
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
import tensorflow as tf
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences

import sentencepiece as spm


Using TensorFlow backend.
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bigdata/.conda/envs/py36/l

In [4]:
class Attention(Layer):
    def __init__(self, regularizer = None, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.regularizer = regularizer
        self.supports_masking = True

    def build(self, input_shape):
        self.context = self.add_weight(name='context', shape=(input_shape[-1], 1),initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),regularizer=self.regularizer,trainable=True)
        super(Attention, self).build(input_shape)
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        attention_in = K.exp(K.squeeze(K.dot(x, self.context), axis=-1))
        attention = attention_in/K.expand_dims(K.sum(attention_in, axis=-1), -1)

        if mask is not None:
        # use only the inputs specified by the mask
        # import pdb; pdb.set_trace()
            attention = attention*K.cast(mask, 'float32')

        weighted_sum = K.batch_dot(K.permute_dimensions(x, [0, 2, 1]), attention)
        return weighted_sum

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [5]:
! nvidia-smi

Mon Jul  6 16:19:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   30C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   27C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [7]:
petition_data = pd.read_csv('petition.csv', encoding='utf-8')

In [8]:
def title_content_combine(title,content):
    return title + ' ' + str(content)

In [9]:
petition_data['petition'] = petition_data.apply(lambda x : title_content_combine(x.title, x.content), axis = 1)

In [10]:
data = petition_data[['petition', 'category']]

In [11]:
data.groupby(['category']).size().sort_values(ascending=False)

category
정치개혁           62750
기타             48682
인권/성평등         34993
안전/환경          30638
교통/건축/국토       28184
외교/통일/국방       26626
육아/교육          25537
보건복지           24848
일자리            23225
행정             20268
문화/예술/체육/언론    18207
미래             18000
경제민주화          16793
성장동력            7253
반려동물            4057
저출산/고령화대책       3584
농산어촌            1902
dtype: int64

In [107]:
max_nb_instances = 100000
x  = pd.DataFrame(data.petition)
y = pd.DataFrame(data.category)
y.columns = ['category']
x_train = x[:max_nb_instances]
y_train = y[:max_nb_instances]

In [13]:
def preprocess(text):
    text = text.replace('\n', '')
    t = re.sub('[^0-9a-zA-Zㄱ-힗,.!?""''\s]', '', text)
    return t

In [108]:
x['mod_petition'] = x.apply(lambda j : preprocess(j.petition), axis =1)

In [109]:
x_train = x.mod_petition[:max_nb_instances]
y_train = pd.DataFrame(y_train)

In [96]:
reviews = []
texts = []
max_length_text = 0
for text in x.mod_petition[:max_nb_instances]:
    sentences =kss.split_sentences(text)
    max_length_text = max(len(sentences), max_length_text)
    for sen in sentences:
        texts.append(sen)
    reviews.append(sentences)
print(max_length_text)

727


In [17]:
with open('spm_input.txt', 'w', encoding='utf-8') as f:
    for sent in texts:
        f.write('{}\n'.format(sent))
    f.close()

In [18]:
with open('spm_input2.txt', 'w', encoding='utf-8') as f:
    for sent in texts:
        f.write('{}\n'.format(sent))
    f.close()

In [19]:
sp = spm.SentencePieceProcessor()

In [20]:
sp.load('Hierarchical_Attention_Networks_Test/300k_vocab.model')

True

In [103]:
MAX_SENTENCES = 10
MAX_SENTENCE_LENGTH = 25
max_length_words =0 
X_train = np.zeros((max_nb_instances, MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
result = []
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            result.append(sp.EncodeAsPieces(sent))
            wordTokens = sp.EncodeAsIds(sent)
            max_length_words = max(len(wordTokens), max_length_words)
            for k in range(len(wordTokens)):
                if k< MAX_SENTENCE_LENGTH:
                    X_train[i][j][k] = wordTokens[k]

In [104]:
def category_to_number(y_data):
    category_list = list(set(y_data.category))
    new_y = []
    for i, category in enumerate(y_data.category):
        new_y.append(category_list.index(category))
    ret = pd.DataFrame(new_y)
    ret.columns = ['category']
    return ret,category_list

In [110]:
y_train.columns = ['category']
Y_train,category_list = category_to_number(y_train)

In [111]:
Y_train.groupby(['category']).size().sort_values(ascending=False)

category
16    17810
9     12632
2     10097
5      8459
0      8450
4      5982
15     5524
10     5018
13     4914
11     4288
12     4171
7      4048
8      3945
6      2454
1       931
14      898
3       379
dtype: int64

In [112]:
train_sample = pd.DataFrame(Y_train)
train_sample.columns = ['y']
train_sample_y = train_sample.groupby('y').size().reset_index()
train_sample_y.columns = ['y', 'cnt']
train_sample_y
train_sample_y['s_cnt'] = np.log2(train_sample_y.cnt)
train_sample_y




,y,cnt,s_cnt
0,0,8450,13.044736
1,1,931,9.862637
2,2,10097,13.301639
3,3,379,8.566054
4,4,5982,12.546412
5,5,8459,13.046271
6,6,2454,11.260920
7,7,4048,11.982994
8,8,3945,11.945810
9,9,12632,13.624795


In [113]:
train_x = []
train_y = []

for idx, row in train_sample_y.iterrows():
    cand_x = X_train[(np.random.choice(Y_train[Y_train.y ==row.y].index, 100*int(row.s_cnt)))]
    cand_y = [row.y]*(int(row.s_cnt)*100)
    
    train_x = train_x + list(cand_x)
    train_y = train_y + cand_y

In [114]:
train_y = to_categorical(np.asarray(train_y))

In [49]:
train_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [115]:
validation_split = 0.1
nb_validation_samples = int(validation_split * len(train_x))

In [116]:
x_train = train_x[:-nb_validation_samples]
y_train = train_y[:-nb_validation_samples]
x_val = train_x[-nb_validation_samples:]
y_val = train_y[-nb_validation_samples:]
x_train, x_val, y_train, y_val = train_test_split(
    train_x, train_y, test_size=0.1, random_state=42)

In [53]:
with open('Hierarchical_Attention_Networks_Test/300k_vocab.vocab', encoding='utf-8') as f:
    Vo = [doc.strip().split("\t") for doc in f]
    
word2idx = {w[0]: i for i, w in enumerate(Vo)}

In [54]:
word_index = list(word2idx.keys())

In [55]:
len(word_index)

300000

In [102]:
len(x_train[0][0])

100

In [58]:
w2v_model.wv.most_similar('대통령')

[('▁대통령', 0.7808446884155273),
 ('▁여당', 0.7431647777557373),
 ('탄핵', 0.7428842782974243),
 ('▁문죄인', 0.7347427010536194),
 ('촛불', 0.7334913015365601),
 ('▁각료', 0.7313647866249084),
 ('민주당', 0.7306030988693237),
 ('▁임명한', 0.7303953170776367),
 ('▁대통령도', 0.7269872426986694),
 ('한국당', 0.7248755693435669)]

In [57]:
embedding_dim = 100
w2v_model = Word2Vec(result, size=embedding_dim, window=5, workers=4, sg=1, iter = 5)

In [117]:
def get_vector(word, word2vec_model):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [118]:
embedding_matrix = np.zeros((len(list(word_index))+1, embedding_dim))
for word, i in enumerate(list(word2idx.keys())):
    temp = get_vector(word, w2v_model)
    if temp is not None:
        embedding_matrix[i] = temp

/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  


In [119]:
l2_reg = regularizers.l2(1e-8)

sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
word_encoder = Bidirectional(GRU(100, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
dense_transform_w = Dense(100,  name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
word_attention_model = attention_weighted_sentence


texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
sentence_encoder = Bidirectional(GRU(100, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
dense_transform_s = Dense(100,  name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
prediction = Dense(y_train.shape[1], activation='softmax')(attention_weighted_text)
model = Model(texts_in, prediction)

model.compile(optimizer=keras.optimizers.SGD(momentum=0.9),loss='categorical_crossentropy',metrics=['acc'])


In [120]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 10, 25)            0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 10, 100)           30140900  
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 10, 200)           120600    
_________________________________________________________________
dense_transform_s (Dense)    (None, 10, 100)           20100     
_________________________________________________________________
sentence_attention (Attentio (None, 100)               100       
_________________________________________________________________
dense_8 (Dense)              (None, 17)                1717      
Total params: 30,283,417
Trainable params: 30,283,417
Non-trainable params: 0
________________________________________________________________

In [122]:

model.fit(x=[x_train],
          y=[y_train],
          batch_size=32,
          epochs=5,
          shuffle=True,
          validation_data=([x_val], 
                           [y_val]))

Train on 17730 samples, validate on 1970 samples
Epoch 1/5
17730/17730 [==============================] - 60s 3ms/step - loss: 2.7787 - acc: 0.0984 - val_loss: 2.7852 - val_acc: 0.0832
Epoch 2/5
 3744/17730 [=====>........................] - ETA: 45s - loss: 2.7832 - acc: 0.0978

KeyboardInterrupt: 

In [78]:
def normalize(text):
    reviews = []
    texts = []
    return kss.split_sentences(text) 

def encode_input(reviews):
    
    
    
    ret = np.zeros((MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
    for j, sent in enumerate(reviews):
        if j < MAX_SENTENCES:
            wordTokens = sp.EncodeAsIds(sent)
            for k in range(len(wordTokens)):
                if k< MAX_SENTENCE_LENGTH:
                    ret[j][k] = wordTokens[k]
            
    return ret

In [79]:
text1 = '공기업 비정규직의 정규화 그만해주세요. 만 하다가 처음으로 청원 글을 써봅니다. 공기업 비정규직의 정규직 전환 이제 그만해주십시오. 그간 한국도로공사 철도공사 서울교통공사 등등 많은 공기업들이 비정규직 정규화가 이루어졌습니다. 솔직히 비정규직 철폐라는 공약이 앞으로 비정규직 전형을 없애 채용하겠다던지, 해당 직렬의 자회사 정규직인 줄 알았습니다. 현실은 더 하더라구요.무분별한 비정규직의 정규화 당장 그만해야한다고 생각합니다.'
text2 = '6월 12일 오후 1시 발생한 사건입니다. 시장 한복판에서 길고양이 학대사건이 발생했습니다. 임신한 고양이가 매장에 들어 이유만으로 줄에 묶여 던지고 목을 조르고 던졌습니다. 그 결과 그 임신한 고양이는 피를 토하며 괴로워했고 상인이 고양이를 박스에 담아 합니다. 시장 주변에서 그 고양이를 담아간 상자를 발견했다고 하는데 그 고양이를 어찌 했는지 모르는 상황입니다. 시장 **가게 근처 **가게 골목 입니다. 정말 임신한 고양이가 매장에 이유 만으로 던지고 줄로 묶어 목을 조르고 이런 행동에서 인간다운 인간성이 있다는 생각이 드는지 의문입니다. 이런 행동을 하는 사람은 사회에 위협이 될 수 있다고 생각합니다. 정확한 수사를 통해 엄벌에 하는 바램입니다. 최근에 동물 학대 사건과 관련하여 이례적인 판례도 나왔습니다. 이를 발판으로 동물보호법 강화를 통해 생명을 경시하는 일이 발생하지 않았으면 하는 바램입니다.'

In [80]:
normalized_text = normalize(preprocess(text2))
encoded_text = encode_input(normalized_text)

In [103]:
encoded_text

array([[  172,    74,   282, ...,     0,     0,     0],
       [ 1364, 58802, 13982, ...,     0,     0,     0],
       [28649, 52161, 39884, ...,     0,     0,     0],
       ...,
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0]], dtype=int32)

In [137]:
normalized_text = normalize(preprocess(text2))

In [139]:
normalized_text

['6월 12일 오후 1시 발생한 사건입니다.',
 '시장 한복판에서 길고양이 학대사건이 발생했습니다.',
 '임신한 고양이가 매장에 들어 이유만으로 줄에 묶여 던지고 목을 조르고 던졌습니다.',
 '그 결과 그 임신한 고양이는 피를 토하며 괴로워했고 상인이 고양이를 박스에 담아 합니다.',
 '시장 주변에서 그 고양이를 담아간 상자를 발견했다고 하는데 그 고양이를 어찌 했는지 모르는 상황입니다.',
 '시장 가게 근처 가게 골목 입니다.',
 '정말 임신한 고양이가 매장에 이유 만으로 던지고 줄로 묶어 목을 조르고 이런 행동에서 인간다',
 '운 인간성이 있다는 생각이 드는지 의문입니다.',
 '이런 행동을 하는 사람은 사회에 위협이 될 수 있다고 생각합니다.',
 '정확한 수사를 통해 엄벌에 하는 바램입니다.',
 '최근에 동물 학대 사건과 관련하여 이례적인 판례도 나왔습니다.',
 '이를 발판으로 동물보호법 강화를 통해 생명을 경시하는 일이 발생하지 않았으면 하는 바램입니다.']

In [104]:
sp.EncodeAsIds(normalized_text[0])

[172, 74, 282, 33, 2150, 34, 193, 1683, 5779, 3]

In [132]:
word_index[75]

'월'

In [130]:
sp.EncodeAsIds('6월 12일 오후 1시 발생한 사건입니다.')

[184, 75, 240, 32, 2035, 33, 104, 1946, 6773, 3]

In [81]:
normalized_text = normalize(preprocess(text2))
encoded_text = encode_input(normalized_text)

hidden_word_encoding_out = Model(inputs=word_attention_model.input,outputs=word_attention_model.get_layer('dense_transform_w').output)
hidden_word_encodings = hidden_word_encoding_out.predict(encoded_text)
word_context = word_attention_model.get_layer('word_attention').get_weights()[0]
u_wattention = encoded_text*np.exp(np.squeeze(np.dot(hidden_word_encodings, word_context)))





nopad_encoded_text = encoded_text[-len(text):]
nopad_encoded_text = [list(filter(lambda x: x > 0, sentence)) for sentence in nopad_encoded_text]
print(nopad_encoded_text)



reconstructed_texts = [[sp.DecodeIds([int(i)]) for i in sentence] for sentence in nopad_encoded_text]
nopad_wattention = u_wattention[:len(normalized_text)]
nopad_wattention = nopad_wattention/np.expand_dims(np.sum(nopad_wattention, -1), -1)
nopad_wattention = np.array([attention_seq[:len(sentence)] for attention_seq, sentence in zip(nopad_wattention, nopad_encoded_text)])
word_activation_maps = []


for i, att in enumerate(nopad_wattention):
    word_activation_maps.append(list(zip(reconstructed_texts[i], att)))

hidden_sentence_encoding_out = Model(inputs=model.input, outputs=model.get_layer('dense_transform_s').output)
hidden_sentence_encodings = np.squeeze(hidden_sentence_encoding_out.predict(np.expand_dims(encoded_text, 0)), 0)
sentence_context = model.get_layer('sentence_attention').get_weights()[0]
u_sattention = np.exp(np.squeeze(np.dot(hidden_sentence_encodings, sentence_context), -1))
nopad_sattention = u_sattention[-len(normalized_text):]

nopad_sattention = nopad_sattention/np.expand_dims(np.sum(nopad_sattention, -1), -1)

activation_map = list(zip(word_activation_maps, nopad_sattention))


[[172, 74, 282, 33, 2150, 34, 193, 1683, 5779, 3], [1364, 58802, 13982, 57630, 16, 11377, 3], [28649, 52161, 39884, 557, 2282, 447, 15, 7248, 28609, 10857, 66541, 30, 104006, 3], [25, 715, 25, 28649, 2823, 11, 2512, 123355, 34655, 2156, 26715, 16, 27728, 80032, 12430, 23, 3], [1364, 8348, 25, 27728, 12430, 1253, 8, 70201, 5535, 2586, 287, 25, 27728, 1614, 4822, 590, 897, 3], [1364, 2494, 7996, 2494, 14112, 86, 3], [88, 28649, 52161, 39884, 1078, 3823, 28609, 34231, 10227, 10857, 66541, 30, 66, 126472, 22986], [8533, 39283, 16, 280, 367, 22446, 3929, 3], [66, 1193, 38, 513, 1112, 8147, 137, 19, 171, 43, 3], [1462, 926, 155, 1503, 38, 6087, 3], [3629, 2509, 3580, 1950, 1273, 97967, 5394, 13, 3741, 3], [310, 74505, 12599, 6162, 155, 1135, 56517, 255, 3162, 4211, 38, 6087, 3], [], [], []]


In [82]:
activation_map

[([('6', 0.016533692204171873),
   ('월', 0.00711333269249255),
   ('12', 0.027107565125444582),
   ('일', 0.0031721618763818128),
   ('오후', 0.2066711525521484),
   ('1', 0.0032682879938479284),
   ('시', 0.018552340670960298),
   ('발생한', 0.16178025569547247),
   ('사건입니다', 0.5555128328366817),
   ('.', 0.0002883783523983466)],
  0.083333336),
 ([('시장', 0.009526869403662677),
   ('한복판에서', 0.41070306061156353),
   ('길고양이', 0.09765739589590289),
   ('학대사건', 0.40251721681310854),
   ('이', 0.00011175213376730411),
   ('발생했습니다', 0.07946275161691368),
   ('.', 2.0953525081369522e-05)],
  0.083333336),
 ([('임신한', 0.08394352000796979),
   ('고양이가', 0.15283528036356284),
   ('매장에', 0.11686283472365062),
   ('들어', 0.0016320479124730068),
   ('이유만으로', 0.006686415325428009),
   ('줄', 0.0013097404252700791),
   ('에', 4.3951020982217415e-05),
   ('묶여', 0.021237133338607456),
   ('던지고', 0.08382631728535053),
   ('목을', 0.03181174898692896),
   ('조르', 0.19496965914518194),
   ('고', 8.790204196443483e-05),
 

In [83]:
ans = model.predict(np.reshape(encoded_text, (1,)+encoded_text.shape))

In [85]:
ans

array([[0.05934172, 0.05819092, 0.05916221, 0.0580809 , 0.05894235,
        0.05866391, 0.05869587, 0.05889405, 0.05854826, 0.05894068,
        0.05893498, 0.05891773, 0.05885287, 0.05884808, 0.05846177,
        0.05920391, 0.05931978]], dtype=float32)

In [84]:
ans.argsort()[0]

array([ 3,  1, 14,  8,  5,  6, 13, 12,  7, 11, 10,  9,  4,  2, 15, 16,  0])

In [36]:
category_list

['미래',
 '육아/교육',
 '반려동물',
 '경제민주화',
 '농산어촌',
 '행정',
 '보건복지',
 '기타',
 '교통/건축/국토',
 '성장동력',
 '정치개혁',
 '일자리',
 '저출산/고령화대책',
 '외교/통일/국방',
 '인권/성평등',
 '문화/예술/체육/언론',
 '안전/환경']

In [ ]:
yahoo = pd.read_csv('yahoo.csv')

In [ ]:
yahoo_x= pd.DataFrame(yahoo.ko)
yahoo_y = pd.DataFrame(yahoo.label)

In [ ]:
yahoo_reviews = []
yahoo_texts = []
for text in yahoo_x.ko:
    sentences =kss.split_sentences(text)
    for sen in sentences:
        yahoo_texts.append(sen)
    yahoo_reviews.append(sentences)

In [ ]:
tokenizer.fit_on_texts(yahoo_texts)
MAX_SENTENCES = 10
MAX_SENTENCE_LENGTH = 50
yahoo_X_train = np.zeros((max_nb_instances, MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
for i, sentences in enumerate(yahoo_reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENTENCE_LENGTH:
                    X_train[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [ ]:
yahoo_Y_train = to_categorical(np.asarray(yahoo_y))

validation_split = 0.1
nb_validation_samples = int(validation_split * yahoo_X_train.shape[0])

yahoo_x_train = yahoo_X_train[:-nb_validation_samples]
yahoo_y_train = yahoo_Y_train[:-nb_validation_samples]
yahoo_x_val = yahoo_X_train[-nb_validation_samples:]
yahoo_y_val = yahoo_Y_train[-nb_validation_samples:]

In [ ]:
l2_reg = regularizers.l2(1e-8)

sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
word_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
dense_transform_w = Dense(100, activation='relu', name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
word_attention_model = attention_weighted_sentence


texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
sentence_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
dense_transform_s = Dense(100, activation='relu', name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
yahoo_prediction = Dense(yahoo_y_train.shape[1], activation='softmax')(attention_weighted_text)
yahoo_model = Model(texts_in, yahoo_prediction)

yahoo_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['acc'])

In [ ]:

yahoo_model.fit(x=[yahoo_x_train],
          y=[yahoo_y_train],
          batch_size=512,
          epochs=10,
          shuffle=True,
          validation_data=([yahoo_x_val], 
                           [yahoo_y_val]))

In [ ]:
w2v_model2 = Word2Vec.load('ko.bin')

In [ ]:
embedding_dim = 200
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in tokenizer.word_index.items():
    temp = get_vector(word, w2v_model2)
    if temp is not None:
        embedding_matrix[i] = temp